In [1]:
# Library Package
import pandas as pd
import numpy as np
import re

# Folder Path
files = './training/training.csv'

# Regular Expression
regex_wage = r'(\$[0-9,]{2,}[kK]?)'
regex_pday = r'(per day)|(p.d)'
regex_html = r'(<[^>]+>)'

In [33]:
frame = pd.read_csv(files, sep='|')
frame.head()

,content,industries,location,title,type_empl,url,wage
0,Amgen is committed to unlocking the potential ...,Healthcare & Medical,"Sydney, Ryde & Macquarie Park","Transformation, Senior Manager - Data, Analyti...",Full Time,https://www.seek.com.au/job/38884116?type=prom...,NaN
1,"Our client, a respected and leading global ban...",Science & Technology,"Sydney, CBD, Inner West & Eastern Suburbs",Senior Analyst | Customer Insights | Banking J...,Full Time,https://www.seek.com.au/job/38983503?type=stan...,NaN
2,We are looking for an experienced Senior Busin...,Information & Communication Technology,"Sydney, North Shore & Northern Beaches","Senior Business Analyst, Data & Analytics (Cor...",Contract/Temp,https://www.seek.com.au/job/38980384?type=stan...,NaN
3,Senior Data Analysts - check this out! This is...,"Advertising, Arts & Media","Sydney, CBD, Inner West & Eastern Suburbs",Data and Analytics Director Job in Sydney - SEEK,Full Time,https://www.seek.com.au/job/39005312?type=stan...,NaN
4,We'll have you working with Australia's most r...,Accounting,"Sydney, CBD, Inner West & Eastern Suburbs",Forensic Technology/Data Analytics - Senior Co...,Full Time,https://www.seek.com.au/job/38991024?type=stan...,NaN


In [3]:
# test_string = """
# Sydney, CBD, Inner West & Eastern Suburbs
# Salary
# $700,000 - $1000,000 per day
# Work Type
# Contract/Temp
# Classification
# Information & Communication Technology, Developers/Programmers
# """
# ls = re.findall(regex_wage,test_string)
# print(ls)
# ls = [int(wage.strip("$").replace(",","")) for wage in ls]
# print(ls)
# print(np.mean(ls))

In [4]:
# frame.info()

In [5]:
# frame['wage'].unique()

In [ ]:
### Special Cases

# 96,000k
# https://www.seek.com.au/job/39003699?type=standout&searchrequesttoken=1907a61e-aba7-41b1-b94a-37f7f06e0f7c

# 130,000-150,000k
# https://www.seek.com.au/job/38915724?type=standout&searchrequesttoken=d0b07562-538a-480d-ab14-3078fa096fbe

# 90,000k
# https://www.seek.com.au/job/38963567?type=standout&searchrequesttoken=d6ff298d-f248-4cc1-866b-0d7b1c2f1b27

In [37]:
# Split Url out of content
def get_wage(regex, sample):
    result = re.findall(regex, sample)
    per_day = re.search(regex_pday,sample)

    if result:
        # Basic clean.
        result = [value.strip("$").replace(",","").lower() for value in result]
        print("=======================B4: ",result)
        
        # findall might return an duplicate values
        # Reduce the length into 2 first values
        if len(result) >= 2:
            result = result[:2]
            print("I-If: ",result)
            
            # Case-of "k" appears 2times
            if ("k" in result[0]) and ("k" in result[-1]):
                final = int(result[0].strip("k")) + int(result[-1].strip("k"))
                final = final/2
                # Treat special cases
                if final < 1000:
                    final = final*1000
            
            # Case-of "k" appears 1time at last.
            elif ("k" not in result[0]) and ("k" in result[-1]):
                k_value = int(result[-1].strip("k"))
                if k_value < 1000 else k_value]
                print(type(k_value))
                print(type(result[0]))
                
                final = (int(result[0]) + k_value)/2
            
            # Case-of "k" appears 1time at first.
            elif ("k" in result[0]) and ("k" not in result[-1]):
                k_value = int(result[0].strip("k"))
                if k_value < 1000:
                    k_value = k_value*1000
                
                final = (k_value + int(result[-1]))/2
                
            elif len(result[0]) < 5 and len(result[-1]) < 5:
                final = np.mean([int(value)*260 for value in result])
                if not per_day:
                    final = final*7.5
            else:
                final = np.mean([int(value) for value in result])
                
            print("O-If: ",final)
            return final
        else:
            result = result[0]
            print("I-Else: ",result)
            
            if "k" in result:
                k_value = int(result.strip("k"))
                # Treat special case
                if k_value < 1000:
                    result = k_value*1000
                else:
                    result = k_value
            elif len(result) < 5:
                result = int(result)*260
                if not per_day:
                    result = int(result)*7.5
            else:
                result = int(result)
            print("O-Else: ", result)
            print("========\n")
            return result
    else:
        return 0

In [38]:
frame['wage'] = frame['wage'].map(lambda row: get_wage(regex_wage,str(row)))

=======================B4:  ['118513', '137829', '118513', '137829', '118513', '137829', '118513', '137829']
I-If:  ['118513', '137829']
O-If:  128171.0
=======================B4:  ['700', '1000']
I-If:  ['700', '1000']
O-If:  221000.0
=======================B4:  ['118513', '137829', '118513', '137829', '118513', '137829', '118513', '137829']
I-If:  ['118513', '137829']
O-If:  128171.0
=======================B4:  ['160']
I-Else:  160
O-Else:  312000.0

=======================B4:  ['950', '1040']
I-If:  ['950', '1040']
O-If:  258700.0
=======================B4:  ['650', '800']
I-If:  ['650', '800']
O-If:  188500.0
=======================B4:  ['140k']
I-Else:  140k
O-Else:  140000

=======================B4:  ['500']
I-Else:  500
O-Else:  130000

=======================B4:  ['150k']
I-Else:  150k
O-Else:  150000

=======================B4:  ['100000', '119999']
I-If:  ['100000', '119999']
O-If:  109999.5
=======================B4:  ['150000']
I-Else:  150000
O-Else:  150000

============

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [26]:
frame['wage'] = frame['wage'].astype('int64')
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354 entries, 0 to 8353
Data columns (total 7 columns):
content       8054 non-null object
industries    8054 non-null object
location      8054 non-null object
title         8054 non-null object
type_empl     8054 non-null object
url           8354 non-null object
wage          8354 non-null int64
dtypes: int64(1), object(6)
memory usage: 456.9+ KB


In [27]:
frame['wage'].unique()

array([       0,   128171,   221000,   312000,   258700,   188500,
         140000,   130000,   150000,   109999,   135525,   135000,
         390000,   112500,   120000,   145000,   100065,   115000,
         234000,   155000,   102500,   180000,    78000,   218400,
         208000,   200000,   273000,   546000,   149500,   160000,
         169000,   105000,   182000,  1384500,   100000,  1540500,
        1755000,  1218750,   122500,    95550,   155348,    90000,
         227500,  1365000,    91650,   136500,   103120,  1594125,
         156000,    84000,   139100,   114999,    79999,   116000,
        1852500,    85000,    77500,   110000, 75065000,   104000,
         195000,   125000,   176800,  1613625,    57499,   110065,
         165750,    63000,  1267500,   780000,  1511250,    95000,
          84500,   112400,   290000,   152955,  1560000,   120450,
          59800,   101400,   230000,   139999,    55000,   253500,
         175000,    60000,   117000,   146250,     8060,    62

In [31]:
frame[frame['wage'] > 10000000]['url'].iloc[0]

'https://www.seek.com.au/job/38915724?type=standout&searchrequesttoken=d0b07562-538a-480d-ab14-3078fa096fbe'

In [10]:
frame1 = frame[frame['wage'] > 0]
frame1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1764 entries, 6 to 8352
Data columns (total 7 columns):
content       1764 non-null object
industries    1764 non-null object
location      1764 non-null object
title         1764 non-null object
type_empl     1764 non-null object
url           1764 non-null object
wage          1764 non-null int64
dtypes: int64(1), object(6)
memory usage: 110.2+ KB


In [11]:
frame1['location'].value_counts()

Sydney, CBD, Inner West & Eastern Suburbs      660
Melbourne, CBD & Inner Suburbs                 322
Brisbane, CBD & Inner Suburbs                  142
Sydney                                         106
Sydney, North Shore & Northern Beaches         102
Sydney, Parramatta & Western Suburbs            99
Melbourne                                       55
Melbourne, Bayside & South Eastern Suburbs      50
Brisbane                                        34
Sydney, Ryde & Macquarie Park                   34
Melbourne, Western Suburbs                      29
Melbourne, Eastern Suburbs                      26
Sydney, South West & M5 Corridor                23
Melbourne, Northern Suburbs                     23
Sydney, Southern Suburbs & Sutherland Shire     14
Sydney, North West & Hills District             14
Brisbane, Northern Suburbs                      10
Brisbane, Southern Suburbs & Logan              10
Brisbane, Western Suburbs & Ipswich              7
Brisbane, Bayside & Eastern Sub

In [12]:
frame1['industries'].value_counts()

Information & Communication Technology    560
Accounting                                287
Marketing & Communications                154
Banking & Financial Services               96
Sales                                      90
Education & Training                       80
Government & Defence                       73
Healthcare & Medical                       54
Manufacturing, Transport & Logistics       54
Science & Technology                       52
Human Resources & Recruitment              40
Administration & Office Support            39
Engineering                                32
Retail & Consumer Products                 28
Consulting & Strategy                      22
Insurance & Superannuation                 18
Call Centre & Customer Service             16
Construction                               12
Advertising, Arts & Media                  12
Real Estate & Property                     10
Legal                                       8
Community Services & Development  

In [13]:
frame1['content'] = frame1['content'].map(lambda row: re.sub(regex_html,'',row))

/home/shayne/Anaconda3/envs/caps_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
no = 0
print(frame1['url'].iloc[no])
print("======================")
print(frame1['wage'].iloc[no])
print("======================")
frame1['content'].iloc[no]

https://www.seek.com.au/job/38988764?type=standout&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
128171


"Salary from $118,513 to $137,829 (depending on experience) + 15.4% superannuation Flexible location - Sydney or Melbourne CBD Lead a small team, contributing to the regulation of Australia's Wealth Management industry A future with ASIC means that your work will contribute to ASIC's vision for a fair, strong and efficient financial system for all Australians. We value what you will bring. We value those with sharp, analytical minds and are open to challenging the way things are done. The team Wealth Management is a critical part of Australia's financial system and has significant impact on the well-being of Australians. The role of ASIC's Wealth Management Group is to regulate the conduct of investment managers, superannuation funds and financial advisers. The Strategy, Analysis & Operations team provides specialist services to support the work of the Wealth Management Group. It provides strategic guidance on business planning and reporting, management of group performance, budget pla